# Preparing bathymetry and land-sea mask
The land-sea mask is an essential input in `DIVAnd`, as it indicates where the interpolation has to be performed.      
The most straightforward method to build the mask is to rely on a bathymetry.

In [ ]:
using DIVAnd
using Dates
using Statistics
using Makie, CairoMakie, GeoMakie
include("../config.jl")

The full EMODnet Bathymetry is a quite big file (~1GB) that can be obtained from the bathymetry portal: http://portal.emodnet-bathymetry.eu/?menu=19. In the present notebook, we will work with the GEBCO bathymetry with a resolution that has been decreased.

<div class="alert alert-block alert-info">
ℹ️ An extremely fine bathymetry is only necessary when working on small, regional domains.
</div>

## Data download and domain selection
We create a domain encompassing the north of Corsica.     
The bathymetry file is automatically downloaded (if not already present).

In [ ]:
dx = dy = 0.125
lonr = 2.5:dx:12.0
latr = 42.3:dy:44.6
depthr = [0, 10, 20, 500]

bathname = gebco08file
download_check(gebco08file, gebco08fileURL)
bathisglobal = true;

We have two main tools to load the bathymetry:
1. `DIVAnd.extract_bath` loads the bathymetry at the resolution defined in the netCDF file while 
2. `DIVAnd.load_bath` reduces the resolution to match the resolution of the analysis.

## Extract bathymetry

In [ ]:
?extract_bath

In [ ]:
bx, by, b = extract_bath(bathname, true, lonr, latr)
@show size(b);

### Plot

In [ ]:
plot_bathy(bx, by, b)

## Load bathymetry
With a change of its original resolution

In [ ]:
?load_bath

In [ ]:
bx, by, b = load_bath(bathname, true, lonr, latr)

### Plot

In [ ]:
plot_bathy(bx, by, b)

## Mask creation
The land-sea mask delimitates the region where the interpolation will be performed.    
It is a 3 dimensional binary matrix.

### Using a loop on the depth levels
We start by defining the mask as a 3D matrix containing only `false` values, then we set the values to `true` when the actual depth is deeper than the selected values stored in the vector `depthr`.

In [ ]:
mask = falses(size(b, 1), size(b, 2), length(depthr))

for k = 1:length(depthr)
    for j = 1:size(b, 2)
        for i = 1:size(b, 1)
            mask[i, j, k] = b[i, j] >= depthr[k]
        end
    end
end
@show typeof(mask)

The mask can be represented at any depth from the provided list of depth `depthr`.

In [ ]:
NN = 3
plot_mask(bx, by, mask[:, :, NN], depth = depthr[NN])

### Using the `load_mask` function
A more direct solution is to use the `load_mask` function, where we provide the bathymetry, the grid and the depth levels as inputs.

In [ ]:
xmask, ymask, mmask = load_mask(bathname, true, lonr, latr, depthr);

In [ ]:
NN = 3
plot_mask(bx, by, mmask[:, :, NN], depth = depthr[NN])

## Flood filling
This tool allows user to remove small lakes or islands.     
To demonstrate it, we add an artificial make on land.

In [ ]:
mask[3, 3, 1] = true

## Mask editing
To edit the mask, you can either 
- modify directly its values if you know the indices, or
- use a selection based on the coordinates.
### Create an artificial island
We simply modify the mask at a given location.

In [ ]:
# Create a copy of the mask for ediding
mask_copy = copy(mask);
mask_copy[63, 8, :] .= false;

In [ ]:
plot_mask(bx, by, mask_copy[:, :, 1], depth = depthr[1])

### Remove an island (based on indices)

In [ ]:
mask_copy = copy(mask);
mask_copy[48:59, 1:10, :] .= true;

In [ ]:
plot_mask(bx, by, mask_copy[:, :, 1], depth = depthr[1])

### Remove an island (based on coordinates). 

This is the recommended approach as you can change resolution easily
`sel_lon` and `sel_lat` are obtained by checking the longitude and latitudes.

In [ ]:
mask_copy = copy(mask);
sel_lon = (bx .> 8) .& (bx .< 10.0);
sel_lat = (by .< 43.25);
mask_copy[sel_lon, sel_lat, :] .= true;

plot_mask(bx, by, mask_copy[:, :, 1], depth = depthr[1])

### Removing isolated sea pixels
Let's edit the previous mask and add an isolated sea point in the western part of the domain.    

In [ ]:
mask[3, 3, 1] = true
plot_mask(bx, by, mask[:, :, 1], depth = depthr[1])

The function `floofill` can help us identify the position of this pixel:

Let's plot the `label` values as obtained by the execution of `floodfill`:

In [ ]:
label = DIVAnd.floodfill(mask)

fig = Figure()
ga = GeoAxis(
    fig[1, 1];
    dest = "+proj=merc",
    title = "Flood fill",
    xticks = 2.0:1.0:14,
    yticks = 42.0:1.0:45.0,
)
hm = heatmap!(ga, bx, by, label[:, :, 1], interpolate = false)
xlims!(ga, (bx[1], bx[end]))
ylims!(ga, (by[1], by[end]))
Colorbar(fig[2, 1], hm, vertical = false)
fig

We see that the isolated sea point is characterised by `label = 2`,        
hence we can discard it by taking the cells for which `label = 1`.

In [ ]:
mask3 = label .== 1

fig = Figure()
ga = GeoAxis(
    fig[1, 1];
    dest = "+proj=merc",
    title = "Flood fill",
    xticks = 2.0:1.0:14,
    yticks = 42.0:1.0:45.0,
)
hm = heatmap!(ga, bx, by, mask3[:, :, 1], interpolate = false)
xlims!(ga, (bx[1], bx[end]))
ylims!(ga, (by[1], by[end]))
Colorbar(fig[2, 1], hm, vertical = false)
fig

## Exercice
1. Load the bathymetry around the Black Sea.
2. Create a land sea mask for the following levels: 0, 150, 500 and 1000 meters.
3. Remove the Sea of Azov and the Sea of Marmara.
4. Plot the result.

The solution is available in [`2-01-topography-BlackSea`](./solutions/2-01-topography-BlackSea.ipynb) and should look like this:

<img src="./Images/BlackSea_mask.png" width="250px">

## Adapt your own bathymetry (advanced)

To use a local bathymetry, one needs to create a netCDF file which has a similar structure than GEBCO files.
In particular, the netCDF file is expected to have the one-dimensional variables `lon` (longitude in _degrees East_) and `lat` (latitude in _degrees North_), and the two-dimentional array `bat` containing the digital terrain model (negative in water and positive above water). 

The order of the dimensions should be: longitude and then latitude in *Column-major* ordering (or latitude and then longitude if the tool ncdump is used, which is based on *Row-major* ordering).

The module `NCDatasets`, provide the utility function `ncgen` which returns the Julia code that creates a netCDF file with the same metadata:

In [ ]:
using NCDatasets
ncgen(bathname)

In this example, one would need to adapt:
* the title attribute of the NetCDF file 
* the size of the dimensions
* define the variables `nclon`, `nclat` and `ncbat`

With this code, you only have to use the coordinates and the bathymetry read from your own file to generate the new bathymetry file.